In [1]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


compenents

chatmodel

In [2]:

# Make sure the model path is correct for your system!
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    model_path="C:/Users/Taki Academy/taki/mistral-7b-v0.1.Q8_0.gguf",
    n_ctx=2048,
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
  
)



llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from C:/Users/Taki Academy/taki/mistral-7b-v0.1.Q8_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 lla

emebdding model

In [3]:
pip install -qU langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\Taki Academy\taki\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


vector store 

In [5]:
pip install -qU langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [7]:
#indexing 
#loading document 

In [8]:
%pip install -qU langchain-community jq 

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_community.document_loaders import JSONLoader

loader = JSONLoader(
    file_path="C:/Users/Taki Academy/taki/myjson.json",
    jq_schema=".summary",
    text_content=False,
)

load

In [10]:
docs = loader.load()
print(docs[0])



page_content='كان عدد المشتركين في شبكة الاتصالات في الولايات المتحدة الأمريكية 646 مشترك لكل 1,000 ساكن، بينما كان عدد المشتركين في الهاتف القار 488 مشترك لكل 1,000 ساكن، بينما كان عدد المشتركين في التلفزة بالكابل 488 مشترك لكل 1,000 ساكن، بينما كان عدد المشتركين في التلفزة بالكابل 488 مشترك لكل 1,000 ساكن، بينما كان عدد المشتركين في التلفزة بالكابل 488 مشترك لكل 1,000 ساكن، بينما كان عدد المشتركين في التلفزة بالكابل 488 مشترك لكل 1,000 ساكن، بينما كان عدد المشتركين في التلفزة بالكابل 488 مشترك لكل 1,000 ساكن، بينما كان عدد المشتركين في التلفزة بالكابل البصرية والأقمار صناعية في الولايات متحدة الأمريكية في الولايات متحدة الأمريكية في الولايات متحدة الأمريكية في الولايات متحدة الأمريكية في الولايات متحدة الأمريكية في الولايات متحدة الأمريكية في' metadata={'source': 'C:\\Users\\Taki Academy\\taki\\myjson.json', 'seq_num': 1}


splitting documents 

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 1 sub-documents.


Storing documents

In [12]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[0])

037a0a12-0fb0-4f48-aeaa-9ee50fe9160d


2. Retrieval and Generation

In [13]:
# from langchain import hub

# # N.B. for non-US LangSmith endpoints, you may need to specify
# # api_url="https://api.smith.langchain.com" in hub.pull.
# from langsmith import Client
# client = Client(api_key="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")
# prompt = hub.pull("rlm/rag-prompt")

# example_messages = prompt.invoke(
#     {"context": "(context goes here)", "question": "(question goes here)"}
# ).to_messages()

# assert len(example_messages) == 1
# print(example_messages[0].content)

In [14]:
#We'll use LangGraph to tie together the retrieval and generation steps into a single application

In [15]:
# # The state of our application controls what data is input to the application, transferred between steps, and output by the application.
# from langchain_core.documents import Document
# from typing_extensions import List, TypedDict


# class State(TypedDict):
#     question: str
#     context: List[Document]
#     answer: str

In [21]:
question = "عدد المشتركين في شبكة الاتصالات في الولايات المتحدة الأمريكية"
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

retrieved_docs = vector_store.similarity_search(question)
retrieved_docs = vector_store.similarity_search(question, k=2)


docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
prompt = prompt.invoke({"question": question, "context": docs_content})
answer = llm.invoke(prompt)

c:\Users\Taki Academy\taki\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Llama.generate: 773 prefix-match hit, remaining 1 prompt tokens to eval


 عدد المشتركين في شبكة الاتصالات في الولايات المتحدة الأمريكية 646 مشترك لكل 1,000 ساكن، بينما كان عدد المشت

KeyboardInterrupt: 